In [1]:
from core.console import console
console.width = 200
with console.status('importing modules'):
    import torch
    import numpy as np
    from time import time
    from typing import Annotated
    from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
    from core.datasets import DatasetLoader
    from core.args.utils import print_args, strip_kwargs, create_arguments, remove_prefix
    from core.loggers import Logger
    from core.methods.base import MethodBase
    from core.methods.gap import GAP, EdgePrivGAP, NodePrivGAP
    from core.methods.sage import SAGE, EdgePrivSAGE, NodePrivSAGE
    from core.methods.mlp import MLP, PrivMLP
    from core.attacks import AttackBase, LinkStealingAttack, NodeMembershipInference
    from core.utils import seed_everything, confidence_interval
    from torch_geometric.data import Data

supported_methods = {
    'gap-inf':  GAP,
    'gap-edp':  EdgePrivGAP,
    'gap-ndp':  NodePrivGAP,
    'sage-inf': SAGE,
    'sage-edp': EdgePrivSAGE,
    'sage-ndp': NodePrivSAGE,
    'mlp':      MLP,
    'mlp-dp':   PrivMLP
}

supported_attacks = {
    'lsa': LinkStealingAttack,
    'nmi': NodeMembershipInference,
}

def run(device:  Annotated[str,   dict(help='device to use', choices=['cpu', 'cuda'])] = 'cuda',
        use_amp: Annotated[bool,  dict(help='use automatic mixed precision training')] = False,
        seed:    Annotated[int,   dict(help='initial random seed')] = 12345,
        repeats: Annotated[int,   dict(help='number of times the experiment is repeated')] = 1,
        **kwargs
    ):

    seed_everything(seed)

    with console.status('loading dataset'):
        loader_args = strip_kwargs(DatasetLoader, kwargs)
        data_initial = DatasetLoader(**loader_args).load(verbose=True)

    num_classes = data_initial.y.max().item() + 1
    config = dict(**kwargs, seed=seed, repeats=repeats)
    logger_args = strip_kwargs(Logger.setup, kwargs)
    logger = Logger.setup(enabled=False, config=config, **logger_args)

    ### initiallize method ###
    Method = supported_methods[kwargs.pop('method')]
    method_args = strip_kwargs(Method, kwargs, prefix='target_')
    method_args = remove_prefix(method_args, prefix='target_')
    method: MethodBase = Method(
        num_classes=num_classes, 
        device=device,
        use_amp=use_amp,
        **method_args
    )

    ### initialize attack ###
    Attack = supported_attacks[kwargs['attack']]
    attack_args = strip_kwargs(Attack, kwargs)
    attack: AttackBase = Attack(
        method=method, 
        device=device,
        use_amp=use_amp,
        **attack_args
    )

    run_metrics = {}

    ### run experiment ###
    for iteration in range(repeats):
        data = Data(**data_initial.to_dict())
        with console.status(f'moving data to {device}'):
            data.to(device)

        start_time = time()
        metrics = attack.execute(data)
        end_time = time()
        metrics['fit_time'] = end_time - start_time

        ### process results ###
        for metric, value in metrics.items():
            run_metrics[metric] = run_metrics.get(metric, []) + [value]
        
        console.print()
        console.info(f'run: {iteration + 1}/{repeats}')
        console.info(f'target/train/acc: {run_metrics["target/train/acc"][-1]:.2f}\t average: {np.mean(run_metrics["target/train/acc"]):.2f}')
        console.info(f'target/test/acc: {run_metrics["target/test/acc"][-1]:.2f}\t average: {np.mean(run_metrics["target/test/acc"]):.2f}')
        console.info(f'shadow/train/acc: {run_metrics["shadow/train/acc"][-1]:.2f}\t average: {np.mean(run_metrics["shadow/train/acc"]):.2f}')
        console.info(f'shadow/test/acc: {run_metrics["shadow/test/acc"][-1]:.2f}\t average: {np.mean(run_metrics["shadow/test/acc"]):.2f}')
        console.info(f'attack/test/acc: {run_metrics["attack/test/acc"][-1]:.2f}\t average: {np.mean(run_metrics["attack/test/acc"]):.2f}')
        console.info(f'attack/adv: {run_metrics["attack/adv"][-1]:.4f}\t average: {np.mean(run_metrics["attack/adv"]):.4f}')
        console.print()

        attack.reset_parameters()

    logger.enable()
    summary = {}
    
    for metric, values in run_metrics.items():
        summary[metric + '_mean'] = np.mean(values)
        summary[metric + '_std'] = np.std(values)
        summary[metric + '_ci'] = confidence_interval(values, size=1000, ci=95, seed=seed)
        logger.log_summary(summary)

    logger.finish()
    print()


def main():
    init_parser = ArgumentParser(add_help=False, conflict_handler='resolve')
    method_subparser = init_parser.add_subparsers(dest='method', required=True, title='algorithm to use')

    for method_name, method_class in supported_methods.items():
        method_parser = method_subparser.add_parser(
            name=method_name, 
            help=method_class.__doc__, 
            formatter_class=ArgumentDefaultsHelpFormatter
        )
        attack_subparser = method_parser.add_subparsers(dest='attack', required=True, title='attack to perform')

        for attack_name, attack_class in supported_attacks.items():
            attack_parser = attack_subparser.add_parser(
                name=attack_name,
                help=attack_class.__doc__,
                formatter_class=ArgumentDefaultsHelpFormatter
            )

            # dataset args
            group_dataset = attack_parser.add_argument_group('dataset arguments')
            create_arguments(DatasetLoader, group_dataset)

            # target method args
            group_method = attack_parser.add_argument_group('method arguments')
            create_arguments(method_class, group_method, prefix='target_')

            # attack method args
            group_attack = attack_parser.add_argument_group('attack arguments')
            create_arguments(attack_class, group_attack)
            
            # experiment args
            group_expr = attack_parser.add_argument_group('experiment arguments')
            create_arguments(run, group_expr)
            create_arguments(Logger.setup, group_expr)

    parser = ArgumentParser(parents=[init_parser], formatter_class=ArgumentDefaultsHelpFormatter)
    kwargs = vars(parser.parse_args(
        args='mlp nmi --target-activation relu --target-batch-norm false --target-val-interval 0 --target-hidden-dim 512 --target-num-layers 4'.split()
    ))
    print_args(kwargs, num_cols=4)

    if kwargs['device'] == 'cuda' and not torch.cuda.is_available():
        console.warning('CUDA is not available, proceeding with CPU') 
        kwargs['device'] = 'cpu'

    try:
        start = time()
        run(**kwargs)
        end = time()
        console.info(f'Total running time: {(end - start):.2f} seconds.')
    except KeyboardInterrupt:
        print('\n')
        console.warning('Graceful Shutdown')
    
    if kwargs['device'] == 'cuda':
        gpu_mem = torch.cuda.max_memory_allocated() / 1024 ** 3
        console.info(f'Max GPU memory used = {gpu_mem:.2f} GB\n')

main()

Output()

[13:10:04] INFO     importing modules...done in 8.27 s                                                                                                                                   ]8;id=275785;file:///idiap/temp/sajadmanesh/.tmp/ipykernel_1222277/872041030.py\872041030.py]8;;\:]8;id=501474;file:///idiap/temp/sajadmanesh/.tmp/ipykernel_1222277/872041030.py#18\18]8;;\

           INFO                                                             program arguments                                                                                               ]8;id=693498;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/args/utils.py\utils.py]8;;\:]8;id=510438;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/args/utils.py#184\184]8;;\
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────                                                   
                      method:             mlp         target_learning_rate:    0.01   attack_num_layers:       2     attack_val_interval:  1                                                            
                      attack:             nmi         target_weight_decay:     0.0    attack_activation:       selu  device:               cuda                                                         
                      dataset:            facebook    target_epochs:           100    attack_dropout:          0.0   use_amp:              False                                                        
                      data_dir:           ./datasets  target_batch_size:       full   attack_batch_norm:       True  seed:                 12345                                                        
                      target_hidden_dim:  512         target_full_batch_eval:  True   attack_optimizer:        adam  repeats:              1                                                            
                      target_num_layers:  4           target_val_interval:     0      attack_learning_rate:    0.01  logger:               csv                                                          
                      target_activation:  relu        num_train_nodes:         10000  attack_weight_decay:     0.0   project:              GAP-DEBUG                                                    
                      target_dropout:     0.0         num_val_nodes:           100    attack_epochs:           100   output_dir:           ./output                                                     
                      target_batch_norm:  False       sort_scores:             False  attack_batch_size:       full  debug:                False                                                        
                      target_optimizer:   adam        attack_hidden_dim:       16     attack_full_batch_eval:  True                                                                                     
                     ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────                                                   

Output()

           INFO                                                            dataset stats                                                                                                    ]8;id=768276;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/datasets/loader.py\loader.py]8;;\:]8;id=860015;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/datasets/loader.py#79\79]8;;\
                     ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────                                                          
                      name       nodes    edges       features   classes   mean degree   median degree   train/val/test (%)      baseline (%)                                                           
                     ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────                                                          
                      facebook   26,406   2,117,924   501        6         80.21         62.0            75.02 / 10.00 / 14.98   22.93                                                                  
                     ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────                                                          

...

           DEBUG    pos_entropy: 0.4275, neg_entropy: 0.4340                                                                                                                                   ]8;id=613471;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/attacks/nmi.py\nmi.py]8;;\:]8;id=528414;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/attacks/nmi.py#36\36]8;;\

           DEBUG    attack dataset: 15012 train nodes, 3754 val nodes, 18766 test nodes                                                                                                       ]8;id=174664;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/attacks/base.py\base.py]8;;\:]8;id=215706;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/attacks/base.py#58\58]8;;\

           INFO     constructing attack dataset...done in 146.61 s                                                                                                                            ]8;id=753891;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/attacks/base.py\base.py]8;;\:]8;id=81623;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/attacks/base.py#58\58]8;;\

           INFO     step 3: training attack model                                                                                                                                             ]8;id=199199;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/attacks/base.py\base.py]8;;\:]8;id=358152;file:///remote/idiap.svm/user.active/sajadmanesh/Github/GAP/core/attacks/base.py#61\61]8;;\

Output()